# Review Classification Pipeline - Inference with HuggingFace Model

This notebook uses the fine-tuned HuggingFace model to classify new reviews without retraining.

## Requirements
1. Run the complete training pipeline first (00_colab_complete_pipeline.ipynb)
2. Ensure trained model exists in `models/saved_models/review_classifier_*/`
3. Place your review data in `data/actual/` directory
4. Data should be in CSV or JSON format with 'id' and 'text' columns

## What This Does
- Loads fine-tuned HuggingFace model from `models/saved_models/`
- Loads auxiliary models (toxicity detection, zero-shot classification)
- Combines custom ML predictions with policy violation detection
- Processes reviews from `data/actual/`
- Outputs comprehensive classification results
- Saves results to `results/inference/`

## Model Architecture
- **Fine-tuned Model**: Custom HuggingFace transformer for review classification
- **Policy Detection**: Zero-shot + toxicity analysis for specific violations
- **Dual-layer**: Both models vote; REJECT if either flags the review

## 1. Environment Setup

In [ ]:
# Install required packages (if not already installed)
!pip install -q transformers torch pandas scikit-learn

import pandas as pd
import json
import os
import sys
from datetime import datetime
from pathlib import Path

print("✅ Environment setup complete")
print(f"Current directory: {os.getcwd()}")

## 2. Load Trained Model

In [ ]:
# Check if trained HuggingFace model exists
model_dir = 'models/saved_models'

print("CHECKING TRAINED MODEL")
print("="*30)

# Look for HuggingFace model directories (timestamped folders)
model_folders = []
if os.path.exists(model_dir):
    for item in os.listdir(model_dir):
        item_path = os.path.join(model_dir, item)
        if os.path.isdir(item_path) and item.startswith('review_classifier_'):
            # Check if it contains HuggingFace model files
            hf_files = ['config.json', 'tokenizer.json', 'tokenizer_config.json']
            model_files = ['model.safetensors', 'pytorch_model.bin']  # Either format
            
            has_config = all(os.path.exists(os.path.join(item_path, f)) for f in hf_files)
            has_model = any(os.path.exists(os.path.join(item_path, f)) for f in model_files)
            
            if has_config and has_model:
                model_folders.append(item)

if not model_folders:
    print(f"❌ ERROR: No trained HuggingFace model found in {model_dir}")
    print(f"\nPlease run the training notebook first:")
    print(f"1. Open 00_colab_complete_pipeline.ipynb")
    print(f"2. Run all cells to train the model")
    print(f"3. Ensure model is saved to models/saved_models/")
    print(f"4. Then return to this inference notebook")
    sys.exit()
else:
    # Use the most recent model (last in alphabetical order = most recent timestamp)
    latest_model = sorted(model_folders)[-1]
    model_path = os.path.join(model_dir, latest_model)
    
    print(f"✅ Found trained model: {latest_model}")
    print(f"   Model path: {model_path}")
    
    # Check model files
    model_files = os.listdir(model_path)
    print(f"   Model files: {', '.join(model_files)}")
    
    # Load latest_config.json if it exists for metadata
    config_file = os.path.join(model_dir, 'latest_config.json')
    if os.path.exists(config_file):
        with open(config_file, 'r') as f:
            config_metadata = json.load(f)
        print(f"\nModel Information:")
        print(f"   Training mode: {config_metadata.get('training_mode', 'Unknown')}")
        print(f"   Timestamp: {config_metadata.get('timestamp', 'Unknown')}")
        print(f"   Confidence threshold: {config_metadata.get('confidence_threshold', 0.55)}")
    else:
        print(f"\nUsing model: {latest_model}")

## 3. Load Inference Pipeline

In [ ]:
# Load the HuggingFace trained model and constants
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import numpy as np

print("LOADING TRAINED MODEL")
print("="*30)

try:
    # Load the fine-tuned HuggingFace model
    print(f"Loading fine-tuned model from: {model_path}")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    
    print(f"✅ Fine-tuned model loaded successfully")
    print(f"   Model type: {model.config.model_type}")
    print(f"   Vocab size: {tokenizer.vocab_size}")
    
    # Load auxiliary models for policy detection
    device = 0 if torch.cuda.is_available() else -1
    print(f"   Device: {'GPU' if device == 0 else 'CPU'}")
    
    print(f"\nLoading auxiliary models...")
    toxic_pipeline = pipeline("text-classification", model="unitary/toxic-bert", top_k=None, device=device)
    zshot_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)
    
    print(f"✅ Auxiliary models loaded")
    
    # Define constants (from training pipeline)
    POLICY_CATEGORIES = {
        'NO_ADS': 'No_Ads',
        'IRRELEVANT': 'Irrelevant', 
        'RANT_NO_VISIT': 'Rant_No_Visit',
        'NONE': 'None'
    }
    
    LABELS = {
        'APPROVE': 'APPROVE',
        'REJECT': 'REJECT'
    }
    
    ZERO_SHOT_LABELS = [
        "an advertisement or promotional solicitation for this business (promo code, referral, links, contact to buy)",
        "off-topic or unrelated to this business (e.g., politics, crypto, chain messages, personal stories not about this place)",
        "a generic negative rant about this business without evidence of a visit (short insults, 'scam', 'overpriced', 'worst ever')",
        "a relevant on-topic description of a visit or experience at this business"
    ]
    
    ZERO_SHOT_TO_POLICY = {
        ZERO_SHOT_LABELS[0]: POLICY_CATEGORIES['NO_ADS'],
        ZERO_SHOT_LABELS[1]: POLICY_CATEGORIES['IRRELEVANT'],
        ZERO_SHOT_LABELS[2]: POLICY_CATEGORIES['RANT_NO_VISIT'],
        ZERO_SHOT_LABELS[3]: POLICY_CATEGORIES['NONE']
    }
    
    # Get confidence threshold from config or use default
    confidence_threshold = config_metadata.get('confidence_threshold', 0.55) if 'config_metadata' in locals() else 0.55
    
    print(f"✅ Constants loaded")
    print(f"   Policy categories: {len(POLICY_CATEGORIES)}")
    print(f"   Zero-shot labels: {len(ZERO_SHOT_LABELS)}")
    print(f"   Confidence threshold: {confidence_threshold}")
    print(f"\n🚀 Ready for inference!")
    
    models_loaded = True
    
except Exception as e:
    print(f"❌ Error loading models: {e}")
    import traceback
    traceback.print_exc()
    print(f"\nTroubleshooting:")
    print(f"1. Ensure the training notebook completed successfully")
    print(f"2. Check that model files exist in: {model_path}")
    print(f"3. Verify internet connection for downloading auxiliary models")
    models_loaded = False

## 4. Load Input Data

In [ ]:
# Check for input data in data/actual directory
input_dir = 'data/actual'
os.makedirs(input_dir, exist_ok=True)

print("LOADING INPUT DATA")
print("="*25)

# Look for CSV and JSON files
input_files = []
if os.path.exists(input_dir):
    for file in os.listdir(input_dir):
        if file.endswith(('.csv', '.json')):
            input_files.append(file)

print(f"Available input files in {input_dir}:")
for file in input_files:
    file_path = os.path.join(input_dir, file)
    file_size = os.path.getsize(file_path)
    print(f"   {file} ({file_size} bytes)")

if not input_files:
    print(f"❌ No input files found in {input_dir}")
    print(f"\nTo add your review data:")
    print(f"1. Create a CSV file with columns: 'id', 'text'")
    print(f"2. Or create a JSON file with array of objects: [{'id': 1, 'text': 'review text'}, ...]")
    print(f"3. Place the file in {input_dir}")
    print(f"4. Re-run this cell")
    print(f"\nExample files are already created for you to test with.")

# Let user choose which file to process
if input_files:
    print(f"\nChoose a file to process:")
    for i, file in enumerate(input_files):
        print(f"   {i+1}. {file}")
    
    # For demo, automatically use the first file
    # In practice, you might want to manually specify the file
    selected_file = input_files[0]
    print(f"\nUsing: {selected_file}")
    
    # Load the selected file
    file_path = os.path.join(input_dir, selected_file)
    
    try:
        if selected_file.endswith('.csv'):
            input_data = pd.read_csv(file_path)
        elif selected_file.endswith('.json'):
            with open(file_path, 'r') as f:
                json_data = json.load(f)
            input_data = pd.DataFrame(json_data)
        
        # Validate required columns
        if 'text' not in input_data.columns:
            print(f"❌ Missing required 'text' column")
            print(f"Available columns: {list(input_data.columns)}")
            input_data = None
        else:
            # Add ID column if missing
            if 'id' not in input_data.columns:
                input_data['id'] = range(1, len(input_data) + 1)
            
            print(f"✅ Data loaded successfully")
            print(f"   Reviews to process: {len(input_data)}")
            print(f"   Columns: {list(input_data.columns)}")
            
            # Show preview
            print(f"\nData Preview:")
            for idx, row in input_data.head(3).iterrows():
                text_preview = str(row['text'])[:60] + "..." if len(str(row['text'])) > 60 else str(row['text'])
                print(f"   ID {row['id']}: {text_preview}")
            
            if len(input_data) > 3:
                print(f"   ... and {len(input_data) - 3} more reviews")
    
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        input_data = None
else:
    input_data = None

## 5. Run Inference

In [ ]:
# Define the inference functions
def predict_with_custom_model(text, model, tokenizer):
    """Predict using fine-tuned model"""
    inputs = tokenizer(text, truncation=True, padding=True, max_length=256, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=-1)
        conf = float(probs.max())
        pred = int(probs.argmax())
    return ("REJECT" if pred == 1 else "APPROVE"), conf

def tox_top_label(tox_output):
    """Extract top toxicity label and score"""
    try:
        if isinstance(tox_output, dict):
            candidates = [tox_output]
        elif isinstance(tox_output, list):
            if len(tox_output) and isinstance(tox_output[0], dict):
                candidates = tox_output
            elif len(tox_output) and isinstance(tox_output[0], list):
                candidates = tox_output[0]
            else:
                candidates = []
        else:
            candidates = []
        if not candidates:
            return "NONE", 0.0
        best = max(candidates, key=lambda d: float(d.get("score", 0.0)))
        return best.get("label", "NONE"), float(best.get("score", 0.0))
    except Exception:
        return "NONE", 0.0

def policy_zero_shot_fused(zshot, toxic, text, tau_irrelevant=0.55, tau_rant=0.55, tau_ads=0.70, tox_tau=0.50):
    """Unified policy detection using zero-shot + toxicity"""
    # Zero-shot classification
    zs_res = zshot(text, candidate_labels=ZERO_SHOT_LABELS, 
                   hypothesis_template="This review is {}.", multi_label=True)
    zs = {lab: float(scr) for lab, scr in zip(zs_res["labels"], zs_res["scores"])}
    
    ads = zs.get(ZERO_SHOT_LABELS[0], 0.0)
    irr = zs.get(ZERO_SHOT_LABELS[1], 0.0)
    rant = zs.get(ZERO_SHOT_LABELS[2], 0.0)
    
    # Toxicity gate
    tox_label, tox_score = tox_top_label(toxic(text))
    
    TOX_TO_RANT = {"toxic", "severe_toxic", "obscene", "threat", "insult"}
    TOX_TO_IRRELEVANT = {"identity_hate"}
    
    if tox_label and tox_score >= tox_tau:
        if tox_label in TOX_TO_RANT:
            return LABELS['REJECT'], POLICY_CATEGORIES['RANT_NO_VISIT']
        if tox_label in TOX_TO_IRRELEVANT:
            return LABELS['REJECT'], POLICY_CATEGORIES['IRRELEVANT']
    
    # Policy thresholds
    if max(irr, rant) >= min(tau_irrelevant, tau_rant):
        return LABELS['REJECT'], (POLICY_CATEGORIES['IRRELEVANT'] if irr >= rant else POLICY_CATEGORIES['RANT_NO_VISIT'])
    
    # Ads detection with evidence
    import re
    AD_PATTERNS = [r"https?://", r"\bwww\.", r"\.[a-z]{2,6}\b", r"\b(?:\+?\d[\s\-()]*){7,}\b",
                   r"\bpromo(?:\s*code)?\b", r"\bdiscount\b", r"\bcoupon\b", r"\breferral\b",
                   r"\buse\s*code\b", r"\bwhatsapp\b", r"\bdm\s+(?:me|us)\b"]
    AD_REGEX = re.compile("|".join(AD_PATTERNS), flags=re.IGNORECASE)
    
    has_ads = bool(AD_REGEX.search(text))
    ads_margin = 0.10
    
    if has_ads and (ads >= tau_ads) and (ads >= max(irr, rant) + ads_margin):
        return LABELS['REJECT'], POLICY_CATEGORIES['NO_ADS']
    
    return LABELS['APPROVE'], POLICY_CATEGORIES['NONE']

def process_reviews(input_df):
    """Process reviews using trained model + policy detection"""
    results = []
    
    for _, row in input_df.iterrows():
        text = str(row['text'])
        
        # Use fine-tuned model for custom classification
        custom_label, custom_conf = predict_with_custom_model(text, model, tokenizer)
        
        # Use policy detection for specific violations
        policy_label, policy_category = policy_zero_shot_fused(
            zshot_pipeline, toxic_pipeline, text,
            tau_irrelevant=0.55, tau_rant=0.55, tau_ads=0.70, tox_tau=0.50
        )
        
        # Final decision: if either flags as REJECT, it's rejected
        if custom_label == 'REJECT' or policy_label == 'REJECT':
            final_label = 'REJECT'
            if policy_label == 'REJECT':
                final_category = policy_category
                final_confidence = 0.8  # High confidence for policy violations
            else:
                final_category = 'Custom_ML_Detected'
                final_confidence = custom_conf
        else:
            final_label = 'APPROVE'
            final_category = 'None'
            final_confidence = max(custom_conf, 0.7)
        
        results.append({
            'id': row['id'],
            'text': text,
            'pred_label': final_label,
            'pred_category': final_category,
            'confidence': round(final_confidence, 4),
            'custom_ml_label': custom_label,
            'custom_ml_confidence': round(custom_conf, 4),
            'policy_label': policy_label,
            'policy_category': policy_category
        })
    
    return pd.DataFrame(results)

# Run inference if everything is ready
if models_loaded and input_data is not None and len(input_data) > 0:
    
    print("RUNNING INFERENCE")
    print("="*20)
    print(f"Processing {len(input_data)} reviews...")
    
    try:
        # Run the inference
        results = process_reviews(input_data)
        
        print(f"✅ Inference completed!")
        print(f"   Processed: {len(results)} reviews")
        
        # Create results directory
        results_dir = 'results/inference'
        os.makedirs(results_dir, exist_ok=True)
        
        # Save results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = f"inference_results_{timestamp}.csv"
        results_path = os.path.join(results_dir, results_file)
        
        results.to_csv(results_path, index=False)
        
        print(f"\nRESULTS SUMMARY")
        print("="*20)
        
        # Summary statistics
        approve_count = len(results[results['pred_label'] == 'APPROVE'])
        reject_count = len(results[results['pred_label'] == 'REJECT'])
        avg_confidence = results['confidence'].mean()
        
        print(f"Total reviews: {len(results)}")
        print(f"APPROVE: {approve_count} ({approve_count/len(results)*100:.1f}%)")
        print(f"REJECT: {reject_count} ({reject_count/len(results)*100:.1f}%)")
        print(f"Average confidence: {avg_confidence:.3f}")
        
        # Category breakdown for rejected reviews
        if reject_count > 0:
            print(f"\nREJECT Categories:")
            reject_categories = results[results['pred_label'] == 'REJECT']['pred_category'].value_counts()
            for category, count in reject_categories.items():
                print(f"   {category}: {count} reviews")
        
        # Show detailed results
        print(f"\nDETAILED RESULTS")
        print("="*50)
        
        display_df = results.copy()
        # Truncate text for display
        display_df['text'] = display_df['text'].apply(lambda x: x[:50] + "..." if len(x) > 50 else x)
        
        display_cols = ['id', 'text', 'pred_label', 'pred_category', 'confidence']
        print(display_df[display_cols].to_string(index=False))
        
        print(f"\n✅ Results saved to: {results_path}")
        print(f"\nSUCCESS: Inference complete!")
        print(f"Your reviews have been classified for policy violations.")
        
    except Exception as e:
        print(f"❌ Error during inference: {e}")
        import traceback
        traceback.print_exc()

else:
    print("❌ Cannot run inference")
    if not models_loaded:
        print("   Models not loaded properly")
    if input_data is None or len(input_data) == 0:
        print("   No input data available")
    
    print(f"\nPlease check:")
    print(f"1. Training notebook was run successfully")
    print(f"2. Input data is placed in data/actual/ directory")
    print(f"3. Input data has required 'text' column")

## 6. Results Analysis

In [ ]:
# Advanced analysis of results (if available)
if 'results' in locals() and len(results) > 0:
    
    print("ADVANCED RESULTS ANALYSIS")
    print("="*30)
    
    # Confidence distribution
    high_conf = results[results['confidence'] >= 0.8]
    medium_conf = results[(results['confidence'] >= 0.6) & (results['confidence'] < 0.8)]
    low_conf = results[results['confidence'] < 0.6]
    
    print(f"Confidence Distribution:")
    print(f"   High (≥0.8): {len(high_conf)} reviews ({len(high_conf)/len(results)*100:.1f}%)")
    print(f"   Medium (0.6-0.8): {len(medium_conf)} reviews ({len(medium_conf)/len(results)*100:.1f}%)")
    print(f"   Low (<0.6): {len(low_conf)} reviews ({len(low_conf)/len(results)*100:.1f}%)")
    
    # Policy violations by type
    print(f"\nPolicy Violation Types:")
    category_counts = results['pred_category'].value_counts()
    for category, count in category_counts.items():
        percentage = count / len(results) * 100
        status = "Policy Violation" if category != "None" else "Clean Review"
        print(f"   {category}: {count} reviews ({percentage:.1f}%) - {status}")
    
    # Flag high-risk reviews
    high_risk = results[
        (results['pred_label'] == 'REJECT') & 
        (results['confidence'] >= 0.8)
    ]
    
    if len(high_risk) > 0:
        print(f"\nHIGH-RISK REVIEWS (High confidence violations):")
        for idx, row in high_risk.iterrows():
            text_preview = row['text'][:60] + "..." if len(row['text']) > 60 else row['text']
            print(f"   ID {row['id']}: {row['pred_category']} ({row['confidence']:.3f}) - {text_preview}")
    
    # Export summary report
    summary_report = {
        'timestamp': datetime.now().isoformat(),
        'total_reviews': len(results),
        'approve_count': len(results[results['pred_label'] == 'APPROVE']),
        'reject_count': len(results[results['pred_label'] == 'REJECT']),
        'average_confidence': float(results['confidence'].mean()),
        'high_confidence_count': len(high_conf),
        'category_breakdown': category_counts.to_dict(),
        'high_risk_reviews': len(high_risk)
    }
    
    summary_path = os.path.join(results_dir, f"summary_report_{timestamp}.json")
    with open(summary_path, 'w') as f:
        json.dump(summary_report, f, indent=2)
    
    print(f"\n✅ Summary report saved: {summary_path}")
    
    print(f"\nINFERENCE COMPLETE")
    print(f"Files created:")
    print(f"   {results_path} - Detailed results")
    print(f"   {summary_path} - Summary report")
    
else:
    print("No results available for analysis")
    print("Run the inference cell first to generate results")